In [1]:
import numpy as np 
import os
#import skimage.io as io
#import skimage.transform as trans
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as keras
import glob as glob
from tensorflow import keras as keras
from tensorflow.keras.callbacks import TensorBoard

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
import cv2 as cv
import matplotlib.pyplot as plt
import random
import tensorflow_addons as tfa

# 加载图片

In [4]:
img_3channel_path = '/home/jiangshengjie/TF/tmp/train_aug/'

In [5]:
label_path = '/home/jiangshengjie/TF/tmp/mask_aug/'

In [6]:
img = glob.glob(img_3channel_path+'/*.png')
label = glob.glob(label_path+'/*.png')

In [7]:
len(img)

2352

In [8]:
len(label)

2352

In [9]:
str1 = 'test'
img.sort(key=lambda x: int(x[x.find(str1) + 5:-4]))

In [10]:
img[2000:2010]

['/home/jiangshengjie/TF/tmp/train_aug/test_112523.png',
 '/home/jiangshengjie/TF/tmp/train_aug/Ttest_114588.png',
 '/home/jiangshengjie/TF/tmp/train_aug/test_114588.png',
 '/home/jiangshengjie/TF/tmp/train_aug/Ltest_114588.png',
 '/home/jiangshengjie/TF/tmp/train_aug/Ttest_114590.png',
 '/home/jiangshengjie/TF/tmp/train_aug/test_114590.png',
 '/home/jiangshengjie/TF/tmp/train_aug/Ltest_114590.png',
 '/home/jiangshengjie/TF/tmp/train_aug/Ltest_115398.png',
 '/home/jiangshengjie/TF/tmp/train_aug/test_115398.png',
 '/home/jiangshengjie/TF/tmp/train_aug/Ttest_115398.png']

In [11]:
str2 = 'real'
label.sort(key=lambda x: int(x[x.find(str2) + 5:-4]))

In [12]:
label[2000:2010]

['/home/jiangshengjie/TF/tmp/mask_aug/Ltest_real_112523.png',
 '/home/jiangshengjie/TF/tmp/mask_aug/Ttest_real_114588.png',
 '/home/jiangshengjie/TF/tmp/mask_aug/Ltest_real_114588.png',
 '/home/jiangshengjie/TF/tmp/mask_aug/test_real_114588.png',
 '/home/jiangshengjie/TF/tmp/mask_aug/Ttest_real_114590.png',
 '/home/jiangshengjie/TF/tmp/mask_aug/test_real_114590.png',
 '/home/jiangshengjie/TF/tmp/mask_aug/Ltest_real_114590.png',
 '/home/jiangshengjie/TF/tmp/mask_aug/Ltest_real_115398.png',
 '/home/jiangshengjie/TF/tmp/mask_aug/test_real_115398.png',
 '/home/jiangshengjie/TF/tmp/mask_aug/Ttest_real_115398.png']

# 预处理函数

In [13]:
def read_png(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img,channels=3)
    return img

def read_png_label(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img,channels=1)
    return img

def crop_img(img,mask):
    #concat_img = tf.concat([img,mask],axis=-1)
    img = tf.image.resize(img,(512,512),method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    #crop_img = tf.image.random_crop(concat_img,[256,256,4])
    return img

def resize_img(img,mask):
    resize_img = tf.image.resize(img,(512,512))
    resize_mask = tf.image.resize(mask,(512,512))
    return resize_img,resize_mask
    
def normal(img,mask):
    img = tf.cast(img,tf.float32)
    mask = tf.cast(mask,tf.int32)
    return img,mask

# 数据集制作+数据增广

In [14]:
def load_image_train(img_path,mask_path):
    img = read_png(img_path)
    mask = read_png_label(mask_path)
    img,mask = resize_img(img,mask)
    #if tf.random.uniform(())>0.5:
        #img=tf.image.flip_left_right(img)
        #mask=tf.image.flip_left_right(mask)
    img,mask = normal(img,mask)

    return img,mask

def load_ud_image(img_path,mask_path):
    img = read_png(img_path)
    img = tf.image.flip_left_right(img)
    mask = read_png_label(mask_path)
    mask = tf.image.flip_left_right(mask)
    
    img,mask = resize_img(img,mask)
    #if tf.random.uniform(())>0.5:
        #img=tf.image.flip_left_right(img)
        #mask=tf.image.flip_left_right(mask)
    img,mask = normal(img,mask)
    return img,mask
    
def load_flip_image(img_path,mask_path):
    img = read_png(img_path)
    img = tf.image.flip_up_down(img)
    mask = read_png_label(mask_path)
    mask = tf.image.flip_up_down(mask)
    
    img,mask = resize_img(img,mask)
    #if tf.random.uniform(())>0.5:
        #img=tf.image.flip_left_right(img)
        #mask=tf.image.flip_left_right(mask)
    img,mask = normal(img,mask)
    return img,mask
    
def load_rot_image90(img_path,mask_path):
    img = read_png(img_path)
    img = tf.image.rot90(img)
    mask = read_png_label(mask_path)
    mask = tf.image.rot90(mask)
    
    img,mask = resize_img(img,mask)
    #if tf.random.uniform(())>0.5:
        #img=tf.image.flip_left_right(img)
        #mask=tf.image.flip_left_right(mask)
    img,mask = normal(img,mask)
    return img,mask

def load_rot_image(img_path,mask_path):
    i = float(random.randrange(10,350,10))
    
    img = read_png(img_path)
    img = tfa.image.transform_ops.rotate(img,i/90)
    mask = read_png_label(mask_path)
    mask = tfa.image.transform_ops.rotate(mask,i/90)
    
    img,mask = resize_img(img,mask)
    #if tf.random.uniform(())>0.5:
        #img=tf.image.flip_left_right(img)
        #mask=tf.image.flip_left_right(mask)
    img,mask = normal(img,mask)
    return img,mask

In [15]:
dataset_train = tf.data.Dataset.from_tensor_slices((img,label))
data_number = len(img)
dataset_train = dataset_train.shuffle(buffer_size=data_number)
dataset_fortrain = dataset_train.take(int(0.8*data_number))
dataset_fortest = dataset_train.skip(int(0.8*data_number))

In [16]:
dataset_img=dataset_fortrain.map(load_image_train)
dataset_label=dataset_fortest.map(load_image_train)

In [17]:
for i in range(0,35):
    dataset_train_rotate = dataset_fortrain.map(load_rot_image)
    dataset_img = dataset_img.concatenate(dataset_train_rotate)

/home/jiangshengjie/anaconda3/envs/TF/lib/python3.7/site-packages/tensorflow/python/autograph/impl/api.py:493: UserWarning: fill_value is not supported and is always 0 for TensorFlow < 2.4.0.
  return py_builtins.overload_of(f)(*args)


In [18]:
tf.data.experimental.cardinality(dataset_img)

<tf.Tensor: shape=(), dtype=int64, numpy=67716>

In [19]:
train_number = 67716

In [20]:
tf.data.experimental.cardinality(dataset_label)

<tf.Tensor: shape=(), dtype=int64, numpy=471>

In [21]:
val_number = 471

In [22]:
BATCH_SIZE=8

In [23]:
dataset_label

<MapDataset shapes: ((512, 512, 3), (512, 512, 1)), types: (tf.float32, tf.int32)>

In [24]:
dataset_img = dataset_img.repeat().batch(BATCH_SIZE)
dataset_label = dataset_label.batch(BATCH_SIZE)

# UNet

In [25]:
def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = keras.layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = keras.layers.UpSampling2D((2, 2))(x)
    concat = keras.layers.Concatenate()([us, skip])
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

In [26]:
def UNet():
    f = [16, 32, 64, 128, 256]
    inputs = tf.keras.layers.Input((512, 512, 3))
    
    p0 = inputs
    c1, p1 = down_block(p0, f[0]) #128 -> 64
    c2, p2 = down_block(p1, f[1]) #64 -> 32
    c3, p3 = down_block(p2, f[2]) #32 -> 16
    c4, p4 = down_block(p3, f[3]) #16->8
    
    bn = bottleneck(p4, f[4])
    
    u1 = up_block(bn, c4, f[3]) #8 -> 16
    u2 = up_block(u1, c3, f[2]) #16 -> 32
    u3 = up_block(u2, c2, f[1]) #32 -> 64
    u4 = up_block(u3, c1, f[0]) #64 -> 128
    
    outputs = keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u4)
    model = keras.models.Model(inputs, outputs)
    return model

In [27]:
model = UNet()
#model = tf.keras.models.load_model('UNET_model.h5')

In [28]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 16) 2320        conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 256, 256, 16) 0           conv2d_1[0][0]                   
_______________________________________________________________________________________

In [29]:
TensorBoard = TensorBoard(log_dir='./model',histogram_freq=1,write_grads=True)

In [30]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics = tf.keras.metrics.MeanIoU(2)
              )

In [31]:
EPOCH=100
STEPS_PER_EPOCH = train_number//BATCH_SIZE
VAL_STEPS = val_number//BATCH_SIZE
ckpt_path = './ckpt/'
cp_callback=tf.keras.callbacks.ModelCheckpoint(ckpt_path,monitor='val_loss',save_weights_only=True,save_best_only=False,mode='auto',save_freq=30)

In [32]:
history = model.fit(dataset_img,
                    epochs=EPOCH,
                    steps_per_epoch=STEPS_PER_EPOCH,
                    validation_data=dataset_label,
                    validation_steps=VAL_STEPS,
                    callbacks=[cp_callback,TensorBoard]
                   )

Epoch 1/100
   1/8464 [..............................] - ETA: 6s - loss: 0.3688 - mean_io_u: 0.4971WARNING:tensorflow:From /home/jiangshengjie/anaconda3/envs/TF/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
8464/8464 [==============================] - 2541s 300ms/step - loss: 0.0716 - mean_io_u: 0.4923 - val_loss: 0.0695 - val_mean_io_u: 0.4922
Epoch 2/100
8464/8464 [==============================] - 2501s 296ms/step - loss: 0.0689 - mean_io_u: 0.4923 - val_loss: 0.0676 - val_mean_io_u: 0.4925
Epoch 3/100
1443/8464 [====>.........................] - ETA: 32:56 - loss: 0.0681 - mean_io_u: 0.4923

KeyboardInterrupt: 

In [ ]:
for image_test in img[0:5]:
    image_test = tf.io.read_file(image_test)
    image_test = tf.image.decode_png(image_test)
    image_test = tf.image.resize(image_test,(512,512))
    image_test = tf.cast(image_test,tf.float32)
    image_test = tf.expand_dims(image_test,axis=0)
    image_test = model.predict(image_test)
    image_test = tf.squeeze(image_test,axis=0)
    plt.figure(figsize=(10,10))
    plt.imshow(image_test)
        

In [ ]:
model.save('UNET_model.h5')#保存整个模型
#model = tf.keras.models.load_model('UNET_model.h5')

In [ ]:
weights = model.get_weights()
model.save_weights('UNET_weights.h5')#保存模型权重
#model.load_weights('UNET_weights.5')

In [ ]:
def img_predict(img_path):
    image_test = tf.io.read_file(img_path)
    image_test = tf.image.decode_png(image_test)
    image_test = tf.image.resize(image_test,(512,512))
    image_test = tf.cast(image_test,tf.float32)
    image_test = tf.expand_dims(image_test,axis=0)
    image_test = model.predict(image_test)
    image_test = tf.squeeze(image_test,axis=0)
    return image_test

In [ ]:
img_ori1 = '/home/jianghshengjie/Tensorflow/tmp/img3channel/test_217.png'

In [ ]:
img_ori2 = '/home/jianghshengjie/Tensorflow/tmp/img3channel/test_433.png'

In [ ]:
img_ori3 = '/home/jianghshengjie/Tensorflow/tmp/img3channel/test_959.png'

In [ ]:
img_pred1 = img_predict(img_ori1)

In [ ]:
img_pred2 = img_predict(img_ori2)

In [ ]:
img_pred3 = img_predict(img_ori3)

In [ ]:
img_label1 = '/home/jianghshengjie/Tensorflow/tmp/img1channel1/test_real_217.png'
img_label2 = '/home/jianghshengjie/Tensorflow/tmp/img1channel1/test_real_433.png'
img_label3 = '/home/jianghshengjie/Tensorflow/tmp/img1channel1/test_real_959.png'

In [ ]:
def img_pre1(img_path):
    image = tf.io.read_file(img_path)
    image = tf.image.decode_png(image)
    image = tf.image.resize(image,(512,512))
    image = tf.cast(image,tf.float32)
    return image
def img_pre2(img_path):
    image = tf.io.read_file(img_path)
    image = tf.image.decode_png(image)
    image = tf.image.resize(image,(512,512))
    image = tf.cast(image,tf.int32)
    return image

In [ ]:
plt.subplot(331)
plt.imshow(img_pre1(img_ori1))
plt.subplot(332)
plt.imshow(img_pred1)
plt.subplot(333)
plt.imshow(img_pre2(img_label1))
plt.subplot(334)
plt.imshow(img_pre1(img_ori2))
plt.subplot(335)
plt.imshow(img_pred2)
plt.subplot(336)
plt.imshow(img_pre2(img_label2))
plt.subplot(337)
plt.imshow(img_pre1(img_ori3))
plt.subplot(338)
plt.imshow(img_pred3)
plt.subplot(339)
plt.imshow(img_pre2(img_label3))
plt.subplots_adjust(left=None,bottom=None,right=None,top=None,wspace=-0.3,hspace=0.1)
plt.savefig('Output.png')

In [ ]:
label4 = img_pre2(img_label3)

In [ ]:
np.unique(label4)